In [ ]:
%pip install -U scikit-learn

In [4]:
import pandas as pd
df = pd.read_csv('/kaggle/input/regression-with-an-insurance-dataset-02/02_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [5]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost

voting_reg = VotingRegressor(
    estimators=[
        ('lin_reg', LinearRegression()),
        ('xgboost', xgb.XGBRegressor(max_depth=6, n_estimators=40, random_state=42)),
        ('catboost',Catboost(iterations=15, depth=6, learning_rate=1, random_state=42))
    ]
)
voting_reg.fit(X_train, y_train)

0:	learn: 1.0731559	total: 147ms	remaining: 2.05s
1:	learn: 1.0678309	total: 229ms	remaining: 1.49s
2:	learn: 1.0666248	total: 307ms	remaining: 1.23s
3:	learn: 1.0641093	total: 389ms	remaining: 1.07s
4:	learn: 1.0639321	total: 456ms	remaining: 913ms
5:	learn: 1.0634042	total: 534ms	remaining: 801ms
6:	learn: 1.0632383	total: 607ms	remaining: 694ms
7:	learn: 1.0631505	total: 671ms	remaining: 587ms
8:	learn: 1.0628459	total: 743ms	remaining: 495ms
9:	learn: 1.0627644	total: 816ms	remaining: 408ms
10:	learn: 1.0626949	total: 893ms	remaining: 325ms
11:	learn: 1.0626371	total: 962ms	remaining: 241ms
12:	learn: 1.0625666	total: 1.03s	remaining: 159ms
13:	learn: 1.0624829	total: 1.1s	remaining: 78.9ms
14:	learn: 1.0623491	total: 1.19s	remaining: 0us


VotingRegressor(estimators=[('lin_reg', LinearRegression()),
                            ('xgboost',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interac...,
                                          learning_rate=None, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=6,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=40,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=42, ...)),
                            ('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7d6840d07580>)])

In [7]:
from sklearn.metrics import root_mean_squared_error
y_pred = voting_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.063975862526113

In [8]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [9]:
y_pred = voting_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.067087255547414

In [10]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/kaggle/input/regression-with-an-insurance-dataset-02/02_test.csv')
del test_set['Premium Amount']
y_pred = voting_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/kaggle/input/regression-with-an-insurance-dataset-02/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('voting_reg_01.csv', index=False)